In [ ]:
%run main.py
%load_ext autoreload
%autoreload 2

!mkdir -p {DATA_DIR} {BERT_DIR} {MODEL_DIR}
s3 = S3()

In [ ]:
if not exists(NE5):
    s3.download(S3_NE5, NE5)
    s3.download(S3_FACTRU, FACTRU)

In [ ]:
if not exists(BERT_VOCAB):
    s3.download(S3_BERT_VOCAB, BERT_VOCAB)
    s3.download(S3_BERT_EMB, BERT_EMB)
    s3.download(S3_BERT_ENCODER, BERT_ENCODER)

In [ ]:
items = list(load_lines(BERT_VOCAB))
words_vocab = BERTVocab(items)
tags_vocab = BIOTagsVocab([PER, LOC, ORG])

In [ ]:
torch.manual_seed(SEED)
seed(SEED)

In [ ]:
config = RuBERTConfig()
emb = BERTEmbedding(
    config.vocab_size, config.seq_len, config.emb_dim,
    config.dropout, config.norm_eps
)
encoder = BERTEncoder(
    config.layers_num, config.emb_dim, config.heads_num, config.hidden_dim,
    config.dropout, config.norm_eps
)
ner = BERTNERHead(config.emb_dim, len(tags_vocab))
model = BERTNER(emb, encoder, ner)

for param in emb.parameters():
    param.requires_grad = False

load_model(model.emb, BERT_EMB)
load_model(model.encoder, BERT_ENCODER)
model = model.to(DEVICE)

In [ ]:
markups = {}
for path, name in [(NE5, TRAIN), (FACTRU, TEST)]:
    lines = load_gz_lines(path)
    items = parse_jl(lines)
    items = log_progress(items, desc=path)
    records = []
    for item in items:
        record = SpanMarkup.from_json(item)
        tokens = list(tokenize(record.text))
        record = record.to_bio(tokens)
        records.append(record)
    markups[name] = records

In [ ]:
encode = BERTNEREncoder(
    words_vocab, tags_vocab,
    seq_len=128,
    batch_size=32,
    shuffle_size=10000
)

batches = {}
for name in [TEST, TRAIN]:
    batches[name] = [_.to(DEVICE) for _ in encode(markups[name])]

In [ ]:
board = Board(BOARD_NAME, RUNS_DIR)
boards = {
    TRAIN: board.section(TRAIN_BOARD),
    TEST: board.section(TEST_BOARD),
}

In [ ]:
optimizer = optim.Adam([
    dict(params=encoder.parameters(), lr=BERT_LR),
    dict(params=ner.parameters(), lr=LR),
])
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, LR_GAMMA)

In [ ]:
meters = {
    TRAIN: NERScoreMeter(),
    TEST: NERScoreMeter(),
}

for epoch in log_progress(range(EPOCHS)):
    model.train()
    for batch in log_progress(batches[TRAIN], leave=False):
        optimizer.zero_grad()
        batch = process_batch(model, ner.crf, batch)
        batch.loss.backward()
        optimizer.step()
    
        score = NERBatchScore(batch.loss)
        meters[TRAIN].add(score)

    meters[TRAIN].write(boards[TRAIN])
    meters[TRAIN].reset()

    model.eval()
    with torch.no_grad():
        for batch in log_progress(batches[TEST], leave=False, desc=TEST):
            batch = process_batch(model, ner.crf, batch)
            batch.target = split_masked(batch.target.value, batch.target.mask)
            batch.pred = ner.crf.decode(batch.pred.value, batch.pred.mask)
            score = score_ner_batch(batch, tags_vocab)
            meters[TEST].add(score)

        meters[TEST].write(boards[TEST])
        meters[TEST].reset()
    
    scheduler.step()
    board.step()

In [ ]:
dump_model(model.encoder, MODEL_ENCODER)
dump_model(model.ner, MODEL_NER)
        
s3.upload(MODEL_ENCODER, S3_MODEL_ENCODER)
s3.upload(MODEL_NER, S3_MODEL_NER)